In [12]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
import requests


In [13]:
response = requests.get('https://datos.madrid.es/egob/GET/catalogo/209434-0-templos-otros.json')
jsons_1 = response.json()
jsons_1.keys()
json_trabajado = jsons_1['@graph']

In [14]:
df_flat_1 = pd.json_normalize(json_trabajado)

In [15]:
responses = requests.get('https://datos.madrid.es/egob/GET/catalogo/209426-0-templos-catolicas.json')
json = responses.json()

In [16]:
json_trabajado2 = json['@graph']

In [17]:
df_flat = pd.json_normalize(json_trabajado2)

In [18]:
df_final = pd.concat([df_flat, df_flat_1], axis=0)

In [19]:
df_final

,@id,@type,id,title,relation,address.district.@id,address.area.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.accesibility,organization.schedule,organization.services,organization.organization-name
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,3371,Basílica Pontificia de San Miguel,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28005,CALLE SAN JUSTO 4,40.414203,-3.709672,"Metro: Sol (líneas 1, 2 y 3), Tirso de Molin...",3,,,Basílica Pontificia de San Miguel
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,3557,Basílica Jesús de Medinaceli,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28014,PLAZA JESUS 2,40.414018,-3.695608,"Metro: Antón Martín (línea 1), Banco de Espa...",3,,,Basílica Jesús de Medinaceli
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,3394,Basílica La Milagrosa,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28010,CALLE GARCIA DE PAREDES 45,40.436738,-3.695725,"Metro: Alonso Cano (línea 7), Gregorio Marañ...",3,,,Basílica La Milagrosa
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11242837,Capilla de San Isidro,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28005,CALLE AGUILA 1,40.410035,-3.711938,Descripción monumental: el Oratorio está ubic...,1,"Día 4 de cada mes (si coincide con un sábado, ...",,Capilla de San Isidro
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,3361,Catedral Santa María la Real de la Almudena,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28013,CALLE BAILEN 10,40.415548,-3.713670,Su construcción comienza a finales del siglo X...,3,,,Catedral Santa María la Real de la Almudena
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11731432,Testigos Cristianos de Jehová. Calle Sierra Sa...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28031,CALLE SIERRA SALVADA 24 LOCAL 5,40.373674,-3.618489,"Metro: Alto del Arenal (1) Bus: 58, 103, 14...",3,,,Testigos Cristianos de Jehová. Calle Sierra Sa...
151,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11432588,Testigos Cristianos de Jehová. Calle Tembleque,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28024,CALLE TEMBLEQUE 64 PLANTA BAJA,40.395189,-3.760301,"Metro: Empalme, Campamento (línea 5) Bus: 2...",3,,,Testigos Cristianos de Jehová. Calle Tembleque
152,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,189351,Testigos Cristianos de Jehová. Calle Venancio ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28038,CALLE VENANCIO MARTIN 19 PLANTA BAJA 2,40.397957,-3.661471,Metro: Nueva Numancia (línea 1) Bus: 141 B...,3,,,Testigos Cristianos de Jehová. Calle Venancio ...
153,https://datos.madrid.es/

In [ ]:
monumetos = df_final.drop(columns = ["@id", "@type", "id",  ])

In [21]:
connect_str = 'mysql+pymysql://ironhack_user:%Vq=c>G5@173.201.189.217/BiciMAD'
engine = create_engine(connect_str)
inspector = inspect(engine)

In [22]:
query_1 = '''

SELECT *
FROM bicimad_stations
'''

df = pd.read_sql_query(query_1, connect_str)

In [23]:
df[["longitud","latitud"]]=df["geometry.coordinates"].str.split(",",expand=True)
df['longitud'] = df['longitud'].str.replace('[','',regex=True)
df["latitud"]=df['latitud'].str.replace(']','',regex=True)

df['latitud'] = df['latitud'].astype(float)
df['longitud'] = df['longitud'].astype(float)
df

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,longitud,latitud
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",-3.701834,40.417214
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]",-3.701603,40.417313
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]",-3.705842,40.420589
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]",-3.706917,40.430294
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]",-3.702587,40.428552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]",-3.729970,40.438960
260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]",-3.726990,40.443750
261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]",-3.726930,40.443420
262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]",-3.727295,40.448332


In [31]:
from shapely.geometry import Point
import geopandas as gpd   # conda install -c conda-forge geopandas

def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

In [32]:
def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
        start = to_mercator(lat_start, long_start)
        finish = to_mercator(lat_finish, long_finish)
        return start.distance(finish)


KeyError: 'latitud'